# Model Yükleme ve Confidence Analizi

Bu notebook, eğitilmiş BERT modelini yükleyip her label için en yüksek confidence değerine sahip adresi bulur.

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
from collections import defaultdict
import os

print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Ti


In [2]:
# --- 2. Model ve Tokenizer Yükleme ---
model_path = r"C:\Users\bicer\OneDrive\Masaüstü\hepsiburada-hackathon-kaggle-etabi\son_model_tam_veri"

print("Model yükleniyor...")
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Label Encoder'ı yükle
label_encoder_path = os.path.join(model_path, "label_encoder.pkl")
label_encoder = joblib.load(label_encoder_path)

print(f"✅ Model yüklendi! Sınıf sayısı: {model.config.num_labels}")
print(f"✅ Label encoder yüklendi! Label sayısı: {len(label_encoder.classes_)}")

# GPU'ya taşı
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()


Model yükleniyor...
✅ Model yüklendi! Sınıf sayısı: 10390
✅ Label encoder yüklendi! Label sayısı: 10390
✅ Model yüklendi! Sınıf sayısı: 10390
✅ Label encoder yüklendi! Label sayısı: 10390


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [3]:

# --- 3. Veri Yükleme ---
data_path = r"C:\Users\bicer\Downloads\preprocesswithoutno.csv"
print(f"Veri yükleniyor: {data_path}")

df = pd.read_csv(data_path)
print(f"✅ Veri yüklendi! Toplam kayıt: {len(df):,}")
print(f"✅ Benzersiz label sayısı: {df['label'].nunique()}")


Veri yükleniyor: C:\Users\bicer\Downloads\preprocesswithoutno.csv
✅ Veri yüklendi! Toplam kayıt: 848,172
✅ Benzersiz label sayısı: 10390
✅ Veri yüklendi! Toplam kayıt: 848,172
✅ Benzersiz label sayısı: 10390


In [4]:
# --- 4. Her Label İçin En Yüksek Confidence'lı 10 Adresi Bulma ---
def find_top_representative_addresses(model, tokenizer, df, label_encoder, device='cuda', top_k=10):
    """Her label için en yüksek confidence'a sahip top K adresi bulur"""
    
    model.eval()
    label_to_top_addresses = {}
    
    unique_labels = df['label'].unique()
    total_labels = len(unique_labels)
    
    print(f"\n🔍 {total_labels} label için top {top_k} address analizi başlıyor...")
    print("=" * 60)
    
    for idx, label in enumerate(unique_labels):
        # Her 1000 label'da bir progress göster
        if (idx + 1) % 1000 == 0 or idx == 0:
            progress = ((idx + 1) / total_labels) * 100
            print(f"📊 İşlenen: {idx + 1}/{total_labels} ({progress:.1f}%)")
        
        # Bu label'a ait adresleri filtrele
        label_data = df[df['label'] == label].copy()
        addresses = label_data['address'].tolist()
        
        if len(addresses) == 0:
            continue
        
        # Performans için sample (isteğe bağlı - büyük label'lar için)
        if len(addresses) > 2000:
            import random
            addresses = random.sample(addresses, 2000)
        
        # Label encoder kontrolü
        try:
            expected_class = label_encoder.transform([label])[0]
        except:
            continue
        
        try:
            with torch.no_grad():
                # Tüm adresleri tek seferde tokenize et
                encodings = tokenizer(
                    addresses, 
                    truncation=True, 
                    padding=True, 
                    max_length=128,
                    return_tensors='pt'
                )
                
                # GPU'ya gönder
                inputs = {k: v.to(device) for k, v in encodings.items()}
                
                # Forward pass - tek seferde tüm adresler
                outputs = model(**inputs)
                logits = outputs.logits
                
                # Softmax ile probability'lere çevir
                probs = torch.softmax(logits, dim=-1)
                
                # Bu label için confidence skorları
                label_confidences = probs[:, expected_class].cpu().numpy()
                
                # En yüksek confidence'lı top K indeksleri bul
                top_indices = np.argsort(label_confidences)[-top_k:][::-1]  # Büyükten küçüğe
                
                # Top K adresleri ve confidence'ları kaydet
                top_addresses = []
                for i in top_indices:
                    if i < len(addresses):  # Index kontrolü
                        top_addresses.append({
                            'address': addresses[i],
                            'confidence': float(label_confidences[i])
                        })
                
                # Eğer yeterli adres yoksa mevcut olanları al
                while len(top_addresses) < top_k:
                    top_addresses.append({
                        'address': '',  # Boş adres
                        'confidence': 0.0
                    })
                
                label_to_top_addresses[label] = top_addresses[:top_k]
                
                # Memory temizleme
                del inputs, outputs, logits, probs, label_confidences
                
        except Exception as e:
            # Hata durumunda boş liste ekle
            label_to_top_addresses[label] = [{'address': '', 'confidence': 0.0} for _ in range(top_k)]
            continue
        
        # Her 5000 label sonrası cache temizle
        if (idx + 1) % 5000 == 0:
            torch.cuda.empty_cache()
    
    print(f"\n🎉 Analiz tamamlandı! {len(label_to_top_addresses)}/{total_labels} label başarıyla işlendi.")
    return label_to_top_addresses

# Analizi çalıştır
print("\n🚀 Top 10 adres analizi başlıyor...")
top_addresses_data = find_top_representative_addresses(model, tokenizer, df, label_encoder, device, top_k=10)


🚀 Top 10 adres analizi başlıyor...

🔍 10390 label için top 10 address analizi başlıyor...
📊 İşlenen: 1/10390 (0.0%)
📊 İşlenen: 1000/10390 (9.6%)
📊 İşlenen: 1000/10390 (9.6%)
📊 İşlenen: 2000/10390 (19.2%)
📊 İşlenen: 2000/10390 (19.2%)
📊 İşlenen: 3000/10390 (28.9%)
📊 İşlenen: 3000/10390 (28.9%)
📊 İşlenen: 4000/10390 (38.5%)
📊 İşlenen: 4000/10390 (38.5%)
📊 İşlenen: 5000/10390 (48.1%)
📊 İşlenen: 5000/10390 (48.1%)
📊 İşlenen: 6000/10390 (57.7%)
📊 İşlenen: 6000/10390 (57.7%)
📊 İşlenen: 7000/10390 (67.4%)
📊 İşlenen: 7000/10390 (67.4%)
📊 İşlenen: 8000/10390 (77.0%)
📊 İşlenen: 8000/10390 (77.0%)
📊 İşlenen: 9000/10390 (86.6%)
📊 İşlenen: 9000/10390 (86.6%)
📊 İşlenen: 10000/10390 (96.2%)
📊 İşlenen: 10000/10390 (96.2%)

🎉 Analiz tamamlandı! 10390/10390 label başarıyla işlendi.

🎉 Analiz tamamlandı! 10390/10390 label başarıyla işlendi.


In [5]:
# --- 5. Top 10 Adresleri Yan Yana CSV Formatında Kaydetme ---

print("\n🔄 Top 10 adresler CSV formatına dönüştürülüyor...")
print(f"📊 İşlenecek label sayısı: {len(top_addresses_data)}")

# CSV için sütun başlıklarını oluştur
columns = ['label']
for i in range(1, 11):
    columns.extend([f'address_{i}', f'confidence_{i}'])

# Veri listesi oluştur
csv_data = []

print("📝 Veriler hazırlanıyor...", end=" ")
for label, top_addresses in top_addresses_data.items():
    row = [label]  # Label ile başla
    
    # Her bir top address için address ve confidence ekle
    for addr_data in top_addresses:
        row.append(addr_data['address'])
        row.append(addr_data['confidence'])
    
    csv_data.append(row)

print("✅")

print("📊 DataFrame oluşturuluyor...", end=" ")
results_df = pd.DataFrame(csv_data, columns=columns)
results_df = results_df.sort_values('label')
print("✅")

# CSV'ye kaydet
output_path = r"C:\Users\bicer\OneDrive\Masaüstü\top_10_addresses_per_label.csv"
print("💾 CSV'ye kaydediliyor...", end=" ")
results_df.to_csv(output_path, index=False, encoding='utf-8')
print("✅")

print(f"\n🎉 Analiz tamamlandı!")
print(f"📁 Sonuçlar kaydedildi: {output_path}")
print(f"📊 Toplam label: {len(results_df)}")
print(f"📈 CSV formatı: label, address_1, confidence_1, address_2, confidence_2, ..., address_10, confidence_10")

# İlk 3 label'ın ilk 3 adresini göster
if len(results_df) > 0:
    print(f"\n🏆 İLK 3 LABEL'IN İLK 3 ADRESİ:")
    print("-" * 100)
    for idx in range(min(3, len(results_df))):
        row = results_df.iloc[idx]
        print(f"\nLabel {row['label']}:")
        for i in range(1, 4):  # İlk 3 adres
            addr_col = f'address_{i}'
            conf_col = f'confidence_{i}'
            if addr_col in row and row[addr_col]:
                addr_display = row[addr_col][:60] + "..." if len(str(row[addr_col])) > 60 else row[addr_col]
                print(f"  {i}. {row[conf_col]:.4f} - {addr_display}")
            else:
                print(f"  {i}. Boş")

print(f"\n🎯 Dosya yolu: {output_path}")


🔄 Top 10 adresler CSV formatına dönüştürülüyor...
📊 İşlenecek label sayısı: 10390
📝 Veriler hazırlanıyor... ✅
📊 DataFrame oluşturuluyor... ✅
💾 CSV'ye kaydediliyor... ✅

🎉 Analiz tamamlandı!
📁 Sonuçlar kaydedildi: C:\Users\bicer\OneDrive\Masaüstü\top_10_addresses_per_label.csv
📊 Toplam label: 10390
📈 CSV formatı: label, address_1, confidence_1, address_2, confidence_2, ..., address_10, confidence_10

🏆 İLK 3 LABEL'IN İLK 3 ADRESİ:
----------------------------------------------------------------------------------------------------

Label 1:
  1. 0.9999 - erdal çerçi caddesi no 85 milas sebahattin akyüz fen lisesi ...
  2. 0.9997 - erdal çerçi caddesi efekent sitesi 83 l fen lisesini geçince...
  3. 0.9996 - aydınlık evler mahallesi toki lojmanları 7 a

Label 2:
  1. 0.9997 - aydınlık evler mahallesi akkent caddesi no 57 milas muğla ay...
  2. 0.9997 - aydınlıkevler mahallesi aydınlık evler mahallesi akkent site...
  3. 0.9996 - aydınlıkevler mahallesi akkent caddesi no 67/142 a

Label 3